In [1]:
import sounddevice as sd
import matplotlib.pyplot as plt
import numpy as np
import os
from scipy.io.wavfile import write, read
import whisper
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import soundfile as sf
import googletrans

os.chdir(os.getcwd() + '\\recording')

print(sd.query_devices(device=None, kind=None))

   0 Microsoft Sound Mapper - Input, MME (2 in, 0 out)
>  1 Microphone Array (Realtek(R) Au, MME (2 in, 0 out)
   2 Microsoft Sound Mapper - Output, MME (0 in, 2 out)
<  3 Speakers (Realtek(R) Audio), MME (0 in, 2 out)
   4 Primary Sound Capture Driver, Windows DirectSound (2 in, 0 out)
   5 Microphone Array (Realtek(R) Audio), Windows DirectSound (2 in, 0 out)
   6 Primary Sound Driver, Windows DirectSound (0 in, 2 out)
   7 Speakers (Realtek(R) Audio), Windows DirectSound (0 in, 2 out)
   8 Realtek ASIO, ASIO (2 in, 2 out)
   9 Speakers (Realtek(R) Audio), Windows WASAPI (0 in, 2 out)
  10 Microphone Array (Realtek(R) Audio), Windows WASAPI (2 in, 0 out)
  11 Headphones 1 (Realtek HD Audio 2nd output with HAP), Windows WDM-KS (0 in, 2 out)
  12 Headphones 2 (Realtek HD Audio 2nd output with HAP), Windows WDM-KS (0 in, 2 out)
  13 PC Speaker (Realtek HD Audio 2nd output with HAP), Windows WDM-KS (2 in, 0 out)
  14 Microphone Array (Realtek HD Audio Mic Array input), Windows WDM-KS (2 

In [5]:
def record():
    """
    Method to record the audio signal from the computer's microphone (5 seconds)
    """
    Fs = 8000  # Sampling frequency
    duration =5 # Recording duration in seconds
    recording = sd.rec(frames=duration * Fs, samplerate=Fs, channels=1)  # Capture the voice
    print('start')
    sd.wait()
    print('end')

    # save file
    filename = str(os.getcwd()) + "\\audio.wav"
    write(filename= filename, rate = Fs, data=recording)

    return filename


file = record()

start
end


AttributeError: 'str' object has no attribute 'dtype'

In [15]:
def transcribe(filename):
    """
    Transcribe content of one specific .wav file
    """
    #filename = filename.split("\\")[-2] + "\\" + filename.split("\\")[-1]
    model = whisper.load_model("base")
    result = model.transcribe(filename)
    return result["text"]

transcription = transcribe(file)
print("transcription: " + transcription)

C:\Users\pietr\anaconda3\envs\gdsc\lib\site-packages\whisper\transcribe.py:78: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


transcription:  We will find another video that you guys can find from here on out.


In [30]:
translator = googletrans.Translator()


def record_continuosly():
    """
    Method to record the audio signal from the computer's microphone continuosly
    """
    Fs = 8000  # Sampling frequency
    duration =5 # Recording duration in seconds
    recording = sd.rec(frames=duration * Fs, samplerate=Fs, channels=1)  # Capture the voice
    for i in range(5):
        print('start')
        sd.wait()
        print('end')
        filename = str(os.getcwd()) + "\\audio.wav"
        write(filename= filename, rate = Fs, data=recording)
        transcription = transcribe(filename)
        
        translation = translator.translate(transcription)
        
        print(transcription)
        print(translation)


file = record_continuosly()

start
end


AttributeError: 'NoneType' object has no attribute 'group'

In [19]:
def translate(filename):
    """
    Translate audio file from french to english text
    """
    # load model and processor
    processor = WhisperProcessor.from_pretrained("openai/whisper-medium")
    model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-medium")

    # load dummy dataset and read soundfiles
    #ds = load_dataset("common_voice", "fr", split="test", streaming=True)
    #ds = ds.cast_column("audio", Audio(sampling_rate=16_000))
    #input_speech = next(iter(ds))["audio"]["array"]

    # tokenize
    input_features = processor(filename, return_tensors="pt").input_features 
    forced_decoder_ids = processor.get_decoder_prompt_ids(language = "fr", task = "translate")

    predicted_ids = model.generate(input_features, forced_decoder_ids = forced_decoder_ids)
    translation = processor.batch_decode(predicted_ids, skip_special_tokens = True)
    return translation

    
translation = translate(file)
print('translation:', translation)


It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


translation:
[" I have to do some tests to see if it works. It's good, it's not bad."]


In [21]:
def detect_language(filename):
    """
    Method to detect language.
    It will need to be used for translating from the right language into english.
    """

    model = whisper.load_model("base")

    # load audio and pad/trim it to fit 30 seconds
    audio = whisper.load_audio(filename)
    audio = whisper.pad_or_trim(audio)

    # make log-Mel spectrogram and move to the same device as the model
    mel = whisper.log_mel_spectrogram(audio).to(model.device)

    # detect the spoken language
    _, probs = model.detect_language(mel)
    language = {max(probs, key=probs.get)}

    # decode the audio
    options = whisper.DecodingOptions(fp16 = False)
    result = whisper.decode(model, mel, options)

    # print the recognized text
    print(result.text)
    return language


language = detect_language(file)
print("Detected language: ", str(language))

TypeError: expected str, bytes or os.PathLike object, not ndarray